In [3]:
import pandas as pd
import numpy as np

# ---- CycleNet: Feature Engineering for PCB Assembly Cycle Time Prediction ----
# --- Role: Feature Engineer / Domain Expert ---

# ---- Step 0: File paths (Windows compatible) ----
dataset_path = r"C:\Users\ADMIN\Downloads\pcb_cycle_dataset_core.csv"
output_path  = r"C:\Users\ADMIN\Downloads\pcb_cycle_dataset_core_engineered.csv"

# ---- Step 1: Load dataset ----
df = pd.read_csv(dataset_path)
display(df.head())  # Show initial rows to check schema

# ---- Step 2: Document important features (based on domain expertise) ----
important_features = [
    "component_density",    # calculated below
    "operator_experience",  # should exist in data
    "machine_type",         # categorical encoding
    "shift",                # categorical encoding
    "pcb_area"              # used in derived features
]
# (Add or remove features based on actual dataset and expert input)

# ---- Step 3: Feature Engineering Function ----
def create_features(df):
    # Feature: component_density = num_components / pcb_area
    if "num_components" in df.columns and "pcb_area" in df.columns:
        df["component_density"] = df["num_components"] / df["pcb_area"]
    
    # Lag Feature: Previous cycle_time for each machine
    if "cycle_time" in df.columns and "machine_id" in df.columns and "timestamp" in df.columns:
        df = df.sort_values(by=["machine_id", "timestamp"])
        df["prev_cycle_time"] = df.groupby("machine_id")["cycle_time"].shift(1)
        df["prev_cycle_time"].fillna(df["cycle_time"].mean(), inplace=True)
    
    # Encode categorical columns
    for col in ["machine_type", "shift"]:
        if col in df.columns:
            df[col] = df[col].astype("category").cat.codes
    
    return df

# ---- Step 4: Apply feature engineering ----
df_engineered = create_features(df)
display(df_engineered.head())  # Preview engineered features

# ---- Step 5: Save engineered dataset for modeling ----
df_engineered.to_csv(output_path, index=False)
print("Feature engineering complete. Important features included:")
print(important_features)
print(f"Engineered data saved to: {output_path}")

# ---- Step 6: Optional descriptive statistics ----
display(df_engineered.describe(include="all"))

# ---- Step 7: Optional Visualization (Uncomment if needed) ----
# import matplotlib.pyplot as plt
# feature_cols = [c for c in df_engineered.columns if c != 'cycle_time']
# df_engineered[feature_cols].hist(figsize=(14,8))
# plt.tight_layout()
# plt.show()

# ---- Documentation Summary ----
# ## CycleNet: Feature Engineering for PCB Assembly Cycle Time Prediction
# Steps performed:
# 1. Key features identified in collaboration with engineers
# 2. Derived component density and lag cycle time features
# 3. Categorical columns encoded numerically
# 4. Engineered dataset saved for ML model development
# 5. Notebook ready for handoff to ML Engineer

       

,num_components,board_layers,component_density,machine_type,operator_experience,shift,cycle_time
0,152,6,0.87,A,7,Day,85.4
1,229,4,4.44,A,1,Day,121.8
2,142,6,2.98,B,5,Day,76.4
3,64,6,1.24,C,8,Night,48.4
4,156,4,2.35,B,4,Day,80.8


,num_components,board_layers,component_density,machine_type,operator_experience,shift,cycle_time
0,152,6,0.87,0,7,0,85.4
1,229,4,4.44,0,1,0,121.8
2,142,6,2.98,1,5,0,76.4
3,64,6,1.24,2,8,1,48.4
4,156,4,2.35,1,4,0,80.8


Feature engineering complete. Important features included:
['component_density', 'operator_experience', 'machine_type', 'shift', 'pcb_area']
Engineered data saved to: C:\Users\ADMIN\Downloads\pcb_cycle_dataset_core_engineered.csv


,num_components,board_layers,component_density,machine_type,operator_experience,shift,cycle_time
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,178.296000,4.904000,2.830180,0.996000,4.894000,0.524000,101.394600
std,71.772802,2.313761,1.257244,0.815668,2.514639,0.499924,36.276245
min,50.000000,2.000000,0.510000,0.000000,1.000000,0.000000,19.000000
25%,113.750000,2.000000,1.847500,0.000000,3.000000,0.000000,73.775000
50%,179.000000,4.000000,2.910000,1.000000,5.000000,1.000000,98.900000
75%,239.000000,8.000000,3.892500,2.000000,7.000000,1.000000,128.925000
max,299.000000,8.000000,4.990000,2.000000,9.000000,1.000000,201.800000
